In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [8]:
movies = pd.read_csv(r"D:\Project3\data\movies.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_10660\913066160.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(r"D:\Project3\data\movies.csv")


In [9]:
movies.head(5)

,id,name,year,rating,certificate,duration,genre,votes,gross_income,directors_id,directors_name,stars_id,stars_name,description
0,tt4710316,Best in Sex: 2015 AVN Awards,(2015 TV Special),4.0,TV-MA,94 min,"Adult, News",124.0,0,nm1624094,Gary Miller,"nm4766272,nm2670531,nm4920605,nm6284246","Farrah Laurel Abraham,Asa Akira,Anikka Albrite...",The hottest adult stars and top adult movies a...
1,tt1281857,Naughty Novelist,(2008 Video),3.8,Not Certified,88 min,Adult,174.0,0,nm0045256,John Bacchus,"nm0128986,nm1969196,nm0451160,nm6130462","Darian Caine,Jackie Stevens,A.J. Khan,Arrora",Darian is a successful journalist but when she...
2,tt2294954,2011 AVN Awards Show,(2011 TV Special),5.7,Not Certified,83 min,"Adult, News",39.0,0,"nm1624094,nm0754845","Gary Miller,Timothy E. Sabo","nm2200343,nm2670531,nm1267549,nm3585599","Aubrey Addams,Asa Akira,Monique Alexander,Rave...",Add a Plot
3,tt6843596,Best in Sex: 2017 AVN Awards,(2017 TV Special),4.9,TV-MA,87 min,"Adult, News",225.0,0,nm1624094,Gary Miller,"nm5221471,nm2670531,nm4920605,nm3038816","Amirah Adara,Asa Akira,Anikka Albrite,Britney ...","Considered the industry's highest honor, the A..."
4,tt3705604,AVN Awards 2014,(2014 TV Special),6.7,R,82 min,"Adult, News",101.0,0,nm1624094,Gary Miller,"nm2670531,nm4920605,nm6284246,nm3992720","Asa Akira,Anikka Albrite,August Ames,Jessie An...",Erotic filmmaking superstars Chanel Preston an...


In [10]:
movies.shape

(2590932, 14)

In [11]:
movies_small = movies.head(4000)

In [12]:
movies_small.shape

(4000, 14)

In [13]:
movies_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              4000 non-null   object 
 1   name            4000 non-null   object 
 2   year            4000 non-null   object 
 3   rating          4000 non-null   float64
 4   certificate     4000 non-null   object 
 5   duration        4000 non-null   object 
 6   genre           4000 non-null   object 
 7   votes           4000 non-null   object 
 8   gross_income    4000 non-null   object 
 9   directors_id    4000 non-null   object 
 10  directors_name  4000 non-null   object 
 11  stars_id        4000 non-null   object 
 12  stars_name      4000 non-null   object 
 13  description     4000 non-null   object 
dtypes: float64(1), object(13)
memory usage: 437.6+ KB


# Tiền xử lý dữ liệu

In [14]:
# Kiểm tra các giá trị thiếu
print(movies_small.isnull().sum())

id                0
name              0
year              0
rating            0
certificate       0
duration          0
genre             0
votes             0
gross_income      0
directors_id      0
directors_name    0
stars_id          0
stars_name        0
description       0
dtype: int64


In [15]:
# Kiểm tra các giá trị trùng lặp
print(movies_small.duplicated().sum())

0


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Các đặc trưng văn bản thường có các dấu hiệu như sau:

- Chứa các đoạn văn bản dài hoặc ngắn.
- Có định dạng là chuỗi ký tự.
- Thường chứa các từ khóa hoặc cụm từ có ý nghĩa cụ thể liên quan đến nội dung (ví dụ: mô tả phim, thể loại phim, tên đạo diễn, tên diễn viên).
- Trong tập dữ liệu này, các cột như description, genre, directors_name, stars_name là các đặc trưng văn bản vì chúng chứa thông tin dạng chuỗi ký tự mô tả về phim.

In [17]:
# Gộp các cột genres, stars, description, director lại với nhau
movies_small['tags'] = movies_small['genre'] + ' ' + movies_small['stars_name'] + ' ' + movies_small['directors_name'] + ' ' + movies_small['description'] 


C:\Users\Admin\AppData\Local\Temp\ipykernel_10660\1892384614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_small['tags'] = movies_small['genre'] + ' ' + movies_small['stars_name'] + ' ' + movies_small['directors_name'] + ' ' + movies_small['description']


In [18]:
new_df = movies_small[['name', 'tags']]
new_df.head(10)

,name,tags
0,Best in Sex: 2015 AVN Awards,"Adult, News Farrah Laurel Abraham,Asa Akira,An..."
1,Naughty Novelist,"Adult Darian Caine,Jackie Stevens,A.J. Khan,Ar..."
2,2011 AVN Awards Show,"Adult, News Aubrey Addams,Asa Akira,Monique Al..."
3,Best in Sex: 2017 AVN Awards,"Adult, News Amirah Adara,Asa Akira,Anikka Albr..."
4,AVN Awards 2014,"Adult, News Asa Akira,Anikka Albrite,August Am..."
5,Couples Guest House,"Adult, Romance Jayesh Arora,Rakhi Sharma,Ekta ..."
6,Wadd: The Life & Times of John C. Holmes,"Documentary, Adult, Biography John Holmes,Lind..."
7,The Sex Awards,"Adult, News Jessica Drake,Veruca James,Penny P..."
8,Modern Romance,"Adult, Romance Nancy Ace,August Ames,Bill Bail..."
9,The Kama Sutra of Vatsyayana,"Documentary, Adult Mohinderjeet,Tony Montana,C..."


In [19]:
new_df.iloc[1]

name                                     Naughty Novelist
tags    Adult Darian Caine,Jackie Stevens,A.J. Khan,Ar...
Name: 1, dtype: object

In [20]:
# Chuyển về chữ thường và loại bỏ từ dừng
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()  # Chuyển về chữ thường
    word_tokens = word_tokenize(text)  # Tokenize văn bản thành các từ
    # Loại bỏ từ dừng
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)

new_df['tags'] = new_df['tags'].apply(preprocess_text)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10660\1313296879.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(preprocess_text)


In [21]:
new_df['tags'][8]

'adult , romance nancy ace , august ames , bill bailey , vicki chase holly randall add plot'

In [22]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(new_df['tags'])

In [23]:
# Chuyển đổi thành mảng numpy để sử dụng trong mô hình TensorFlow
tfidf_array = tfidf_matrix.toarray()

In [24]:
tfidf_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
X_train, X_test = train_test_split(tfidf_array, test_size=0.2, random_state=42)


In [26]:
# Xây dựng mô hình deep learning
input_dim = tfidf_array.shape[1]

In [27]:
model1 = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(input_dim, activation='sigmoid')
])

# Compile mô hình
model1.compile(optimizer='adam', loss='binary_crossentropy')

# Huấn luyện mô hình
history1 = model1.fit(X_train, X_train, epochs=7, batch_size=32, validation_data=(X_test, X_test))



Epoch 1/7

100/100 [==============================] - 8s 65ms/step - loss: 0.2278 - val_loss: 0.0032
Epoch 2/7
100/100 [==============================] - 5s 54ms/step - loss: 0.0027 - val_loss: 0.0023
Epoch 3/7
100/100 [==============================] - 5s 53ms/step - loss: 0.0022 - val_loss: 0.0021
Epoch 4/7
100/100 [==============================] - 5s 53ms/step - loss: 0.0021 - val_loss: 0.0021
Epoch 5/7
100/100 [==============================] - 6s 56ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 6/7
100/100 [==============================] - 5s 52ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 7/7
100/100 [==============================] - 5s 54ms/step - loss: 0.0020 - val_loss: 0.0021


In [28]:
# Mô hình đơn giản
model2 = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dense(input_dim, activation='sigmoid')
])

# Compile mô hình
model2.compile(optimizer='adam', loss='binary_crossentropy')

# Huấn luyện mô hình
history2 = model2.fit(X_train, X_train, epochs=7, batch_size=32, validation_data=(X_test, X_test))

Epoch 1/7
100/100 [==============================] - 7s 57ms/step - loss: 0.4378 - val_loss: 0.0635
Epoch 2/7
100/100 [==============================] - 5s 52ms/step - loss: 0.0217 - val_loss: 0.0110
Epoch 3/7
100/100 [==============================] - 5s 52ms/step - loss: 0.0065 - val_loss: 0.0060
Epoch 4/7
100/100 [==============================] - 6s 58ms/step - loss: 0.0041 - val_loss: 0.0043
Epoch 5/7
100/100 [==============================] - 6s 63ms/step - loss: 0.0032 - val_loss: 0.0035
Epoch 6/7
100/100 [==============================] - 6s 62ms/step - loss: 0.0027 - val_loss: 0.0031
Epoch 7/7
100/100 [==============================] - 5s 54ms/step - loss: 0.0025 - val_loss: 0.0028


In [ ]:
# Mô hình sâu hơn
model3 = Sequential([
    Dense(256, activation='relu', input_shape=(input_dim,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(input_dim, activation='sigmoid')
])

# Compile mô hình
model3.compile(optimizer='adam', loss='binary_crossentropy')

# Huấn luyện mô hình
history3 = model3.fit(X_train, X_train, epochs=7, batch_size=32, validation_data=(X_test, X_test))

Epoch 1/7
100/100 [==============================] - 8s 62ms/step - loss: 0.2354 - val_loss: 0.0030
Epoch 2/7
100/100 [==============================] - 6s 60ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 3/7
100/100 [==============================] - 6s 59ms/step - loss: 0.0021 - val_loss: 0.0021
Epoch 4/7
100/100 [==============================] - 6s 59ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 5/7
100/100 [==============================] - 6s 60ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 6/7
100/100 [==============================] - 6s 60ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 7/7
 72/100 [====================>.........] - ETA: 1s - loss: 0.0020

In [ ]:
# Mô hình phức tạp hơn
model4 = Sequential([
    Dense(256, activation='relu', input_shape=(input_dim,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(input_dim, activation='sigmoid')
])

# Compile mô hình
model4.compile(optimizer='adam', loss='binary_crossentropy')

# Huấn luyện mô hình
history4 = model4.fit(X_train, X_train, epochs=7, batch_size=32, validation_data=(X_test, X_test))

In [ ]:
# Encode toàn bộ tfidf_array sử dụng mô hình đã huấn luyện
encoded_movies = model2.predict(tfidf_array)

In [ ]:
# Tính toán ma trận độ tương đồng cosine giữa các phim
similarities = cosine_similarity(encoded_movies)

In [ ]:
def recommend_movies(movie_name, similarities, num_recommendations=5):
    if movie_name not in new_df['name'].values:
        return []

    movie_index = new_df[new_df['name'] == movie_name].index[0]
    if movie_index >= len(similarities):
        return []

    similarity_scores = list(enumerate(similarities[movie_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    top_similar_movies_indices = [i[0] for i in similarity_scores[1:num_recommendations+1]]
    
    return new_df['name'].iloc[top_similar_movies_indices]


In [ ]:
# Sử dụng hàm recommend_movies để gợi ý phim
movie_name = "Naughty Novelist"

recommended_movies = recommend_movies(movie_name, similarities)

print("Các bộ phim được gợi ý cho", movie_name, "là:")
print(recommended_movies)

Metrics

In [ ]:
def ndcg_at_k(recommended_movies, relevant_movies, k):
    def dcg_at_k(r, k):
        r = np.asfarray(r)[:k]
        if r.size:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        return 0.0

    r = [1 if movie in relevant_movies else 0 for movie in recommended_movies[:k]]
 
    dcg = dcg_at_k(r, k)
    
    idcg = dcg_at_k(sorted(r, reverse=True), k)
    
    if idcg == 0:
        return 0.0

    return dcg / idcg

def evaluate_ndcg(model, tfidf_array, new_df, num_recommendations=5):
    encoded_movies = model.predict(tfidf_array)
    similarities = cosine_similarity(encoded_movies)

    all_ndcg = []
    for idx, movie_name in enumerate(new_df['name']):
        relevant_movies = new_df['name'][new_df['name'] != movie_name].tolist()
        recommended_movies = recommend_movies(movie_name, similarities, num_recommendations)
        if len(recommended_movies) < num_recommendations:
            continue
        ndcg = ndcg_at_k(recommended_movies, relevant_movies, num_recommendations)
        all_ndcg.append(ndcg)
    return np.mean(all_ndcg)



In [ ]:
def average_precision_at_k(recommended_movies, relevant_movies, k):
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(recommended_movies[:k]):
        if p in relevant_movies:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(relevant_movies), k)

def evaluate_map(model, tfidf_array, new_df, num_recommendations=5):
    encoded_movies = model.predict(tfidf_array)
    similarities = cosine_similarity(encoded_movies)

    all_ap = []
    for idx, movie_name in enumerate(new_df['name']):
        relevant_movies = new_df['name'][new_df['name'] != movie_name].tolist()
        recommended_movies = recommend_movies(movie_name, similarities, num_recommendations)
        if len(recommended_movies) < num_recommendations:
            continue
        ap = average_precision_at_k(recommended_movies, relevant_movies, num_recommendations)
        all_ap.append(ap)
    return np.mean(all_ap)


In [ ]:
# Huấn luyện các mô hình và tính toán NDCG và MAP
ndcg_scores = []
map_scores = []

models = [model1, model2, model3, model4]  # Danh sách các mô hình bạn muốn so sánh

for idx, model in enumerate(models, start=1):
    ndcg_score = evaluate_ndcg(model, tfidf_array, new_df, 5)
    map_score = evaluate_map(model, tfidf_array, new_df,5)
    
    ndcg_scores.append(ndcg_score)
    map_scores.append(map_score)

# Tạo DataFrame để hiển thị kết quả
results_df = pd.DataFrame({
    'Models': ['Model 1','Model 2', 'Model 3', 'Model 4'],  # Tên của các mô hình
    'NDCG': ndcg_scores,
    'MAP': map_scores
})

print("Kết quả đánh giá các mô hình:")
print(results_df)


In [ ]:
import pickle

# Save DataFrame to pickle file
output_file = 'movie_similarity1.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(similarities, f)

# Optionally, also save movie names to another pickle file
movie_names_file = 'movie_names1.pkl'
with open(movie_names_file, 'wb') as f:
    pickle.dump(new_df, f)

In [ ]:
mkdir my-jupyter-notebook